# Redo Marks fitness pred

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
import torch
from math import ceil
sys.path.append("../")
from utils import *
from global_variables import *
from escape_map import *
pgm_path = "PGM/"
sys.path.append(pgm_path + "source/")
sys.path.append(pgm_path + "utilities/")
import utilities, Proteins_utils, sequence_logo, plots_utils
import rbm, RBM_utils

E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 2, 'F', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  dmean_v_dw = np.dot(s1.T, V)
E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 1, 'A', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  mean_V = np.dot(weights, V) / sum_weights
C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Loaded 29 KD vectors
Loaded 2 KD vectors
Loaded 671 KD vectors


In [2]:
# df = pd.read_csv("marks_old.csv")
# dfbis=pd.read_csv("../seq_data/all_mutation_summary.csv")
# dfbis["first_seen_100"] = pd.to_datetime(dfbis["first_seen_100"]).dt.strftime("%Y-%m")

# #first_seen_100 as AAAA-MM instead of AAA-MM-DD
# dfmerged=df.merge(dfbis, on='mutation', how='left')

# dfter=pd.read_csv("../seq_data/all_mutation_summary_old.csv")
# dfter["first_seen"] = pd.to_datetime(dfter["first_seen"]).dt.strftime("%Y-%m")

# dfmerged=dfmerged.merge(dfter, on='mutation', how='left')

# dfmerged.to_csv("rbd_dist_one_scores_gisaid.csv", index=False)

# dfmerged

In [3]:

# --- Load ---
df = pd.read_csv("rbd_dist_one_scores_gisaid.csv")
df=df[df['i']>=349] 
df=df[df['i']<=526]
df.shape



(1401, 24)

In [4]:
Y_ALL = (df["count"] >= 100).to_numpy().astype(int)
scores_baseline=df['evescape'].to_numpy()
#average of y
print(Y_ALL.sum()/len(Y_ALL))

0.185581727337616


In [5]:
WT

'SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG'

In [6]:
def get_sequence(WT, site,	mutation):
    #site is expected to be 349 instead of 0
    site=site-349
    return WT[:site]+mutation+WT[site+1:]

#example:

df['sequence']=df.apply(lambda x: get_sequence(WT, x['i'],  x['mut']), axis=1)
    

In [7]:
with open('single_variants.fasta', 'w') as f:
    for index, row in df.iterrows():
        f.write('>' + str(index) + '\n')
        f.write(row['sequence'] + '\n')
seqs=Proteins_utils.load_FASTA('single_variants.fasta', drop_duplicates=False)
seqs.shape

(1401, 178)

# EscapeMap scoring

In [8]:
from escape_map_torch import EscapeMapTorch
raw_conc = np.full(len(KD_VECTORS), -8.0, dtype=np.float64)  # antibodies
model = EscapeMapTorch(
    rbm=RBM,
    kd_vectors=KD_VECTORS,
    ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=raw_conc,
    raw_ace2=-9,      # ACE2 concentration in log10 space
    raw_beta=-1,
)
scores=-model(seqs)
print('scores shape:', scores)
#auc to compare score and y

auc = roc_auc_score(Y_ALL, scores.detach().numpy())
print("AUC:", auc)
#params are the individual concentrations for each Ab (dont put -12 everywhere), raw_ace2, raw_beta
# i want to find best parameters to maximize predictive power=auc, but could be something else


scores shape: tensor([578.0441, 574.5410, 574.5409,  ..., 576.8531, 576.6968, 572.7727],
       dtype=torch.float64, grad_fn=<NegBackward0>)
AUC: 0.6582569271219577


In [9]:


# # ===== Training with BCE on sigmoid(score - WT) =====
def fit_escape_map_sigmoid_diff(
    seqs, y, rbm, kd_vectors, ace2_vector,
    epochs=50, batch_size=512, lr=2e-1, weight_decay=1e-5,
    device=None, dtype=torch.float64
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs, dtype=torch.long, device=device)
    Y = torch.as_tensor(y.astype(float), dtype=dtype, device=device)

    model = EscapeMapTorch(
        rbm=rbm, kd_vectors=kd_vectors, ace2_vector=ace2_vector,
        total_beta=1.0, device=device, dtype=dtype
    ).to(device)

    # Optimizer: now include raw_concentrations
    params = [model.raw_concentrations, model.raw_beta, model.raw_ace2, model.raw_total_beta]
    opt = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)

    # Class imbalance handling
    pos = float(Y.sum().item()); neg = float(Y.numel() - pos)
    pos_weight = torch.tensor(neg / max(pos, 1.0), dtype=dtype, device=device)
    bce = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    N = X.size(0); idx_all = torch.arange(N, device=device)
    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)

    for _ in range(epochs):
        perm = idx_all[torch.randperm(N, device=device)]
        for bi in range(ceil(N / batch_size)):
            bidx = perm[bi * batch_size : (bi + 1) * batch_size]
            xb, yb = X[bidx], Y[bidx]

            opt.zero_grad()
            wt_score = -model(WT)               # scalar
            scores_b = -model(xb)               # [B]
            logits = scores_b - wt_score        # classification margin vs WT
            loss = bce(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, max_norm=10.0)
            opt.step()
            print('model.raw_beta:', model.raw_beta.item())
            print('model.raw_ace2:', model.raw_ace2.item())

    # Final evaluation
    with torch.no_grad():
        wt_score = -model(WT)
        scores = -model(X)
        logits_full = scores - wt_score
        prob = torch.sigmoid(logits_full).cpu().numpy()
        auc = roc_auc_score(y, prob)
        rc = model.raw_concentrations.detach().cpu().numpy()
        rb = float(model.raw_beta.detach().cpu())
        ra = float(model.raw_ace2.detach().cpu())
        tb = float(torch.exp(model.raw_total_beta).detach().cpu())  # total_beta in linear scale

    return {
        "model": model,
        "prob": prob,
        "AUC": auc,
        "raw_concentrations": rc,
        "raw_beta": rb,
        "raw_ace2": ra,
        "total_beta": tb,
    }


# out = fit_escape_map_sigmoid_diff(
#     seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
#     epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
# )
# print(out["AUC"], out["raw_ace2"], out["raw_beta"], out["total_beta"], out["raw_concentrations"])
# # --- save scores ---
# scores_df = pd.DataFrame({
#     "seq_index": np.arange(len(seqs)),
#     "y": Y_ALL,
#     "score": out["prob"]
# })
# scores_df.to_csv("scores_sigmoid_diff.csv", index=False)

# # --- save parameters ---
# ab_names = list(KD_VECTORS.keys())
# param_row = {
#     "AUC": out["AUC"],
#     "raw_beta": out["raw_beta"],
#     "raw_ace2": out["raw_ace2"],
#     "total_beta": out["total_beta"],
# }
# for name, val in zip(ab_names, out["raw_concentrations"]):
#     param_row[f"raw_c_{name}"] = float(val)

# params_df = pd.DataFrame([param_row])
# params_df.to_csv("params_sigmoid_diff.csv", index=False)




In [10]:


# model = load_escape_map_from_csv("params_sigmoid_diff.csv")
# scores=-model(seqs)
# wt_score = -model(WT_SEQ)
# df['escapemap'] = scores-wt_score.item()
# df

In [11]:


# #group per site i and save mean escapemap score
# df_site=df.groupby('i').agg({'escapemap':'mean', 'evescape':'mean', 'first_seen':'first'}).reset_index()
# df_site.to_csv('site_scores.csv', index=False)
# #print top 10 sites
# df_site.sort_values(by='escapemap', ascending=False).head(10)


In [12]:
import numpy as np
import torch
from math import ceil
from sklearn.metrics import roc_auc_score

# --- stratified subset indices ---
def stratified_subset_indices(y, frac, seed=0):
    y = np.asarray(y).astype(int)
    rng = np.random.default_rng(seed)
    pos = rng.choice(np.flatnonzero(y==1), size=max(1, int(np.ceil(frac*(y==1).sum()))), replace=False)
    neg = rng.choice(np.flatnonzero(y==0), size=max(1, int(np.ceil(frac*(y==0).sum()))), replace=False)
    idx = np.concatenate([pos, neg]); rng.shuffle(idx); return idx

# --- unchanged model class: EscapeMapTorch ---

def fit_escape_map_sigmoid_diff_train_test(
    seqs, y, rbm, kd_vectors, ace2_vector,
    epochs=50, batch_size=512, lr=2e-1, weight_decay=1e-5,
    device=None, dtype=torch.float64, train_idx=None, seed=0
):
    torch.manual_seed(seed); np.random.seed(seed)
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs, dtype=torch.long, device=device)
    Y = torch.as_tensor(np.asarray(y).astype(float), dtype=dtype, device=device)

    # subset for training
    if train_idx is None:
        Xtr, Ytr = X, Y
    else:
        ti = torch.as_tensor(train_idx, dtype=torch.long, device=device)
        Xtr, Ytr = X[ti], Y[ti]

    model = EscapeMapTorch(rbm, kd_vectors, ace2_vector, total_beta=1.0, device=device, dtype=dtype).to(device)
    params = [model.raw_concentrations, model.raw_beta, model.raw_ace2, model.raw_total_beta]
    opt = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)

    # class weight from the subset
    pos = float(Ytr.sum().item()); neg = float(Ytr.numel() - pos)
    pos_weight = torch.tensor(neg / max(pos, 1.0), dtype=dtype, device=device)
    bce = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)
    Ntr = Xtr.size(0); idx_local = torch.arange(Ntr, device=device)

    for _ in range(epochs):
        perm = idx_local[torch.randperm(Ntr, device=device)]
        for bi in range(ceil(Ntr / batch_size)):
            bidx = perm[bi*batch_size : (bi+1)*batch_size]
            xb, yb = Xtr[bidx], Ytr[bidx]
            opt.zero_grad()
            logits = (-model(xb)) - (-model(WT))   # margin vs WT
            loss = bce(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, 10.0)
            opt.step()

    with torch.no_grad():
        logits_full = (-model(X)) - (-model(WT))
        prob = torch.sigmoid(logits_full).cpu().numpy()
        auc = roc_auc_score(np.asarray(y).astype(int), prob)

    return {"model": model, "prob": prob, "AUC": auc,
            "raw_concentrations": model.raw_concentrations.detach().cpu().numpy(),
            "raw_beta": float(model.raw_beta.detach().cpu()),
            "raw_ace2": float(model.raw_ace2.detach().cpu()),
            "total_beta": float(torch.exp(model.raw_total_beta).detach().cpu())}

# --- few-shot runs ---
import pandas as pd

fracs = [0.10, 0.20, 0.30, 0.40, 0.50]

def run_fewshot(kd_vecs, tag, fracs=fracs, seed=42):
    for f in fracs:
        idx = stratified_subset_indices(Y_ALL, frac=f, seed=seed)
        out = fit_escape_map_sigmoid_diff_train_test(
            seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=kd_vecs, ace2_vector=ACE2_KD_VECTOR,
            epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5, train_idx=idx, seed=seed
        )
        print(f"[{tag}] frac={f:.2f} AUC={out['AUC']:.4f}")

        # --- save scores (evaluate on all samples) ---
        scores_df = pd.DataFrame({
            "seq_index": np.arange(len(seqs)),
            "y": np.asarray(Y_ALL).astype(int),
            "score": out["prob"],
            "train_frac": f,
            "ab_set": tag
        })
        scores_df.to_csv(f"scores_sigmoid_diff_{tag}_{int(f*100)}.csv", index=False)

        # --- save parameters ---
        ab_names = list(kd_vecs.keys())
        param_row = {
            "AUC": out["AUC"],
            "raw_beta": out["raw_beta"],
            "raw_ace2": out["raw_ace2"],
            "total_beta": out["total_beta"],
            "train_frac": f,
            "ab_set": tag,
            "n_train": out.get("n_train", len(idx)),
            "pos_train": out.get("pos_train", int(np.asarray(Y_ALL)[idx].sum()))
        }
        for name, val in zip(ab_names, out["raw_concentrations"]):
            param_row[f"raw_c_{name}"] = float(val)

        params_df = pd.DataFrame([param_row])
        params_df.to_csv(f"params_sigmoid_diff_{tag}_{int(f*100)}.csv", index=False)

# run on both antibody sets if desired
# run_fewshot(KD_VECTORS, "allab")
# run_fewshot(kd_vector_small, "4ab")



# Train 80% subset

In [13]:
# import os
# import numpy as np
# import pandas as pd
# from sklearn.metrics import roc_auc_score
# from math import ceil


# # ===== Run 10 splits =====
# os.makedirs("training_80", exist_ok=True)
# N = len(Y_ALL)
# indices = np.arange(N)

# for split in range(10):
#     rng = np.random.RandomState(split)
#     rng.shuffle(indices)
#     train_idx = indices[: N // 5*4].tolist()
#     test_idx  = indices[N // 5*4 :].tolist()

#     out = fit_escape_map_sigmoid_diff_train_test(
#         seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
#         epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
#         train_idx=train_idx,
#     )

#     # save scores
#     test_mask = np.zeros(N, dtype=int)
#     test_mask[test_idx] = 1
#     scores_df = pd.DataFrame({
#         "seq_index": np.arange(N),
#         "y": Y_ALL,
#         "score": out["prob"],
#         "test": test_mask,
#     })
#     split_dir = f"training_80/split_{split}"
#     os.makedirs(split_dir, exist_ok=True)
#     scores_df.to_csv(os.path.join(split_dir, "scores.csv"), index=False)

#     # save parameters
#     ab_names = list(KD_VECTORS.keys())
#     param_row = {
#         "AUC": out["AUC"],
#         "raw_beta": out["raw_beta"],
#         "raw_ace2": out["raw_ace2"],
#         "total_beta": out["total_beta"],
#     }
#     for name, val in zip(ab_names, out["raw_concentrations"]):
#         param_row[f"raw_c_{name}"] = float(val)
#     params_df = pd.DataFrame([param_row])
#     params_df.to_csv(os.path.join(split_dir, "params.csv"), index=False)

#     print(f"Split {split}: AUC={out['AUC']:.3f}, saved in {split_dir}")


In [14]:
df

,Unnamed: 0,i,wt,mut,bloom_ace2_binding,bloom_expression,chan_expression,chan_ace2_binding,fitness_eve,dissimilarity_charge_hydro,...,is_escape_experiment_xie,is_escape_experiment_all,mutation,count_marks,first_seen_marks,count_100,first_seen_100,count,first_seen,sequence
135,135,349,S,A,-0.34,-1.13,-3.83,-2.68,-6.902246,-1.222299,...,False,False,S349A,53.0,NaN,30.0,NaN,30.0,2021-01,AVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
136,136,349,S,C,-0.12,-2.03,-1.77,-0.78,-7.368066,-1.609191,...,False,False,S349C,10.0,NaN,1.0,NaN,1.0,2022-05,CVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
137,137,349,S,F,-2.03,-2.50,-3.46,-2.75,-7.207227,-0.559057,...,False,False,S349F,16.0,NaN,11.0,NaN,11.0,2020-05,FVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
138,138,349,S,G,-0.59,-2.08,-1.04,-0.77,-6.908368,-1.388110,...,False,False,S349G,NaN,NaN,1.0,NaN,1.0,2022-04,GVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
139,139,349,S,I,-4.25,-2.69,-3.35,-1.95,-6.500989,-0.337976,...,False,False,S349I,NaN,NaN,NaN,NaN,NaN,NaN,IVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1531,526,G,E,-0.01,-0.13,NaN,NaN,-8.738202,1.067187,...,False,False,G526E,NaN,NaN,2.0,NaN,2.0,2020-10,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
1532,1532,526,G,R,-0.08,-0.30,NaN,NaN,-8.578857,3.241148,...,False,False,G526R,6.0,NaN,3.0,NaN,3.0,2021-10,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
1533,1533,526,G,S,-0.02,-0.14,NaN,NaN,-8.599188,-1.388110,...,False,False,G526S,NaN,NaN,NaN,NaN,NaN,NaN,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...
1534,1534,526,G,V,-0.04,-0.12,NaN,NaN,-8.664563,-1.461803,...,False,False,G526V,5.0,NaN,1.0,NaN,1.0,2022-01,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...


# Train on beginning of pandemic

In [15]:
import numpy as np
import pandas as pd

# 1) parse the date column
# col = "first seen" if "first seen" in df.columns else "first_seen"
col='first_seen_100'
df["first_seen_dt"] = pd.to_datetime(df[col], format="%Y-%m", errors="coerce")

# 2) 6-month periods starting 2020-01-01
start = pd.Timestamp("2020-01-01")
max_dt = df["first_seen_dt"].max(skipna=True)
if pd.isna(max_dt):
    max_dt = start

# period starts: 2020-01, 2020-07, 2021-01, ...
check_starts = pd.date_range(start, max_dt, freq="3MS")
check_ends = check_starts + pd.DateOffset(months=3)  # label by END date

# 3) y=1 if first_seen < end_of_period (before the end date), else 0
dates = df["first_seen_dt"].values[:, None]                 # (N,1) datetime64[ns]
ends = check_ends.values[None, :]                           # (1,M)
counts=df['count_100'].values[:, None]
valid = (~pd.isna(df["first_seen_dt"]).values)[:, None]     # (N,1) bool
y_mat = (valid & (dates < ends)& (counts>100)).astype(int)                # (N,M)

# 4) attach as columns: y_YYYY-MM (period END date)
y_df = pd.DataFrame(
    y_mat,
    index=df.index,
    columns=[f"y_{d.strftime('%Y-%m')}" for d in check_ends]
)
print("Date range:", start, "to", max_dt)
print("Last date of first_seen:", df["first_seen_dt"].max())

df_out = pd.concat([df, y_df], axis=1)



Date range: 2020-01-01 00:00:00 to 2025-02-01 00:00:00
Last date of first_seen: 2025-02-01 00:00:00


In [16]:
df

,Unnamed: 0,i,wt,mut,bloom_ace2_binding,bloom_expression,chan_expression,chan_ace2_binding,fitness_eve,dissimilarity_charge_hydro,...,is_escape_experiment_all,mutation,count_marks,first_seen_marks,count_100,first_seen_100,count,first_seen,sequence,first_seen_dt
135,135,349,S,A,-0.34,-1.13,-3.83,-2.68,-6.902246,-1.222299,...,False,S349A,53.0,NaN,30.0,NaN,30.0,2021-01,AVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
136,136,349,S,C,-0.12,-2.03,-1.77,-0.78,-7.368066,-1.609191,...,False,S349C,10.0,NaN,1.0,NaN,1.0,2022-05,CVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
137,137,349,S,F,-2.03,-2.50,-3.46,-2.75,-7.207227,-0.559057,...,False,S349F,16.0,NaN,11.0,NaN,11.0,2020-05,FVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
138,138,349,S,G,-0.59,-2.08,-1.04,-0.77,-6.908368,-1.388110,...,False,S349G,NaN,NaN,1.0,NaN,1.0,2022-04,GVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
139,139,349,S,I,-4.25,-2.69,-3.35,-1.95,-6.500989,-0.337976,...,False,S349I,NaN,NaN,NaN,NaN,NaN,NaN,IVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1531,526,G,E,-0.01,-0.13,NaN,NaN,-8.738202,1.067187,...,False,G526E,NaN,NaN,2.0,NaN,2.0,2020-10,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
1532,1532,526,G,R,-0.08,-0.30,NaN,NaN,-8.578857,3.241148,...,False,G526R,6.0,NaN,3.0,NaN,3.0,2021-10,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
1533,1533,526,G,S,-0.02,-0.14,NaN,NaN,-8.599188,-1.388110,...,False,G526S,NaN,NaN,NaN,NaN,NaN,NaN,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT
1534,1534,526,G,V,-0.04,-0.12,NaN,NaN,-8.664563,-1.461803,...,False,G526V,5.0,NaN,1.0,NaN,1.0,2022-01,SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTN...,NaT


In [17]:
y_df.sum(axis=0)

y_2020-04     16
y_2020-07     18
y_2020-10     24
y_2021-01     31
y_2021-04     67
y_2021-07     94
y_2021-10    139
y_2022-01    194
y_2022-04    209
y_2022-07    220
y_2022-10    226
y_2023-01    233
y_2023-04    241
y_2023-07    248
y_2023-10    253
y_2024-01    255
y_2024-04    256
y_2024-07    256
y_2024-10    256
y_2025-01    258
y_2025-04    259
dtype: int64

In [18]:
seqs.shape

(1401, 178)

In [19]:
# import os
# import re
# import numpy as np
# import pandas as pd
# from sklearn.metrics import roc_auc_score

# # fit_escape_map_sigmoid_diff unchanged except that "y" is the period vector you pass in

# _END_RE = re.compile(r"^y_(\d{4})-(\d{2})$")

# def _enddate_sort_key(col: str):
#     m = _END_RE.match(col)
#     if not m:
#         return pd.Timestamp.max
#     return pd.Timestamp(int(m.group(1)), int(m.group(2)), 1)

# def train_all_periods_small_and_save(
#     df_out, seqs, rbm, kd_vecs, ace2_vec,
#     tag, epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
#     out_prefix="sigmoid_diff"
# ):
#     period_cols = sorted([c for c in df_out.columns if c.startswith("y_")], key=_enddate_sort_key)
#     ab_names = list(kd_vecs.keys())
#     os.makedirs("results_periods_smaller_training", exist_ok=True)

#     for col in period_cols:
#         y = df_out[col].to_numpy().astype(int)
#         # skip degenerate periods
#         if y.sum() == 0 or y.sum() == y.size:
#             print(f"skip {col}: single class ({y.sum()}/{y.size})")
#             continue

#         out = fit_escape_map_sigmoid_diff(
#             seqs=seqs, y=y, rbm=rbm, kd_vectors=kd_vecs, ace2_vector=ace2_vec,
#             epochs=epochs, batch_size=batch_size, lr=lr, weight_decay=weight_decay
#         )

#         # AUC may fail if labels collapse numerically
#         try:
#             auc = roc_auc_score(y, out["prob"])
#         except Exception:
#             auc = np.nan

#         period_label = col.replace("y_", "")  # e.g., "2020-07" (END date)

#         # ----- save scores -----
#         scores_df = pd.DataFrame({
#             "seq_index": np.arange(len(seqs)),
#             "y": y,
#             "score": out["prob"],
#             "period": period_label,   # YYYY-MM (END date)
#             "ab_set": tag
#         })
#         scores_path = f"results_periods_smaller_training/scores_{out_prefix}_{tag}_{period_label}.csv"
#         scores_df.to_csv(scores_path, index=False)

#         # ----- save parameters -----
#         param_row = {
#             "period": period_label,
#             "ab_set": tag,
#             "AUC": float(auc),
#             "raw_beta": float(out["raw_beta"]),
#             "raw_ace2": float(out["raw_ace2"]),
#             "total_beta": float(out["total_beta"]),
#         }
#         for name, val in zip(ab_names, out["raw_concentrations"]):
#             param_row[f"raw_c_{name}"] = float(val)

#         params_df = pd.DataFrame([param_row])
#         params_path = f"results_periods_smaller_training/params_{out_prefix}_{tag}_{period_label}.csv"
#         params_df.to_csv(params_path, index=False)

#         print(f"{tag} {col}: AUC={auc:.4f} -> {scores_path}, {params_path}")


# if __name__ == "__main__":
#     train_all_periods_small_and_save(
#         df_out=df_out, seqs=seqs, rbm=RBM, kd_vecs=KD_VECTORS, ace2_vec=ACE2_KD_VECTOR,
#         tag="allab", epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
#         out_prefix="sigmoid_diff"
#     )

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

# fit_escape_map_sigmoid_diff unchanged except that "y" is the period vector you pass in

_END_RE = re.compile(r"^y_(\d{4})-(\d{2})$")

def _enddate_sort_key(col: str):
    m = _END_RE.match(col)
    if not m:
        return pd.Timestamp.max
    return pd.Timestamp(int(m.group(1)), int(m.group(2)), 1)

def train_all_periods_small_and_save(
    df_out, seqs, rbm, kd_vecs, ace2_vec,
    tag, epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
    out_prefix="sigmoid_diff"
):
    period_cols = sorted([c for c in df_out.columns if c.startswith("y_")], key=_enddate_sort_key)
    ab_names = list(kd_vecs.keys())
    os.makedirs("results_periods_smaller_training_large", exist_ok=True)

    for col in period_cols:
        y = df_out[col].to_numpy().astype(int)
        # skip degenerate periods
        if y.sum() == 0 or y.sum() == y.size:
            print(f"skip {col}: single class ({y.sum()}/{y.size})")
            continue

        out = fit_escape_map_sigmoid_diff(
            seqs=seqs, y=y, rbm=rbm, kd_vectors=kd_vecs, ace2_vector=ace2_vec,
            epochs=epochs, batch_size=batch_size, lr=lr, weight_decay=weight_decay
        )

        # AUC may fail if labels collapse numerically
        try:
            auc = roc_auc_score(y, out["prob"])
        except Exception:
            auc = np.nan

        period_label = col.replace("y_", "")  # e.g., "2020-07" (END date)

        # ----- save scores -----
        scores_df = pd.DataFrame({
            "seq_index": np.arange(len(seqs)),
            "y": y,
            "score": out["prob"],
            "period": period_label,   # YYYY-MM (END date)
            "ab_set": tag
        })
        scores_path = f"results_periods_smaller_training_large/scores_{out_prefix}_{tag}_{period_label}.csv"
        scores_df.to_csv(scores_path, index=False)

        # ----- save parameters -----
        param_row = {
            "period": period_label,
            "ab_set": tag,
            "AUC": float(auc),
            "raw_beta": float(out["raw_beta"]),
            "raw_ace2": float(out["raw_ace2"]),
            "total_beta": float(out["total_beta"]),
        }
        for name, val in zip(ab_names, out["raw_concentrations"]):
            param_row[f"raw_c_{name}"] = float(val)

        params_df = pd.DataFrame([param_row])
        params_path = f"results_periods_smaller_training_large/params_{out_prefix}_{tag}_{period_label}.csv"
        params_df.to_csv(params_path, index=False)

        print(f"{tag} {col}: AUC={auc:.4f} -> {scores_path}, {params_path}")


if __name__ == "__main__":
    train_all_periods_small_and_save(
        df_out=df_out, seqs=seqs, rbm=RBM, kd_vecs=KD_VECTORS|BA1_KD_VECTORS, ace2_vec=ACE2_KD_VECTOR,
        tag="allab", epochs=15, batch_size=1024, lr=4e-1, weight_decay=2e-5,
        out_prefix="sigmoid_diff"
    )

model.raw_beta: -1.6000002892551226
model.raw_ace2: -10.399999047966968
model.raw_beta: -1.8899569788835207
model.raw_ace2: -10.799576676044325
model.raw_beta: -2.039305697694754
model.raw_ace2: -11.199118476500477
model.raw_beta: -2.1859075207450207
model.raw_ace2: -11.600544729372842
model.raw_beta: -2.2597468268562486
model.raw_ace2: -12.003117482242672
model.raw_beta: -2.295537403153632
model.raw_ace2: -12.405945516778663
model.raw_beta: -2.2837709297170905
model.raw_ace2: -12.811878330667508
model.raw_beta: -2.229976046721387
model.raw_ace2: -13.212685696961273
model.raw_beta: -2.137429465309539
model.raw_ace2: -13.615584658013276
model.raw_beta: -2.006741766385954
model.raw_ace2: -14.011841007000045
model.raw_beta: -1.8460907044975439
model.raw_ace2: -14.404211640148018
model.raw_beta: -1.6515093846924023
model.raw_ace2: -14.778993483098793
model.raw_beta: -1.4447411898306501
model.raw_ace2: -15.13932755579588
model.raw_beta: -1.201735925025899
model.raw_ace2: -15.476289110183629